<a href="https://colab.research.google.com/github/stiflerGit/CAD-Mammography/blob/master/ENSEMBLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import json
import h5py

# https://github.com/keras-team/keras/issues/11683

def fix_layer0(filename, batch_input_shape, dtype):
    with h5py.File(filename, 'r+') as f:
        model_config = json.loads(f.attrs['model_config'].decode('utf-8'))
        layer0 = model_config['config']['layers'][0]['config']
        layer0['batch_input_shape'] = batch_input_shape
        layer0['dtype'] = dtype
        f.attrs['model_config'] = json.dumps(model_config).encode('utf-8')

# Example
fix_layer0(os.path.join(pre_train_cnn_dir, pre_train_cnn_model_name), [None, 224, 224, 3], 'float32')

{'class_name': 'Sequential', 'config': {'name': 'sequential_1', 'layers': [{'class_name': 'Conv2D', 'config': {'name': 'block1_conv1', 'trainable': False, 'filters': 64, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0, 'mode': 'fan_avg', 'distribution': 'uniform', 'seed': None}}, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None}}, {'class_name': 'Conv2D', 'config': {'name': 'block1_conv2', 'trainable': False, 'filters': 64, 'kernel_size': [3, 3], 'strides': [1, 1], 'padding': 'same', 'data_format': 'channels_last', 'dilation_rate': [1, 1], 'activation': 'relu', 'use_bias': True, 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'scale': 1.0

In [27]:
import os 
from google.colab import drive
from keras import models

drive.mount('/content/drive')

work_dir = "./drive/My Drive/ENSEMBLE/"
if not os.path.exists(work_dir):
  os.makedirs(work_dir)

pre_train_cnn_dir = "./drive/My Drive/PT-CNN"
pre_train_cnn_model_name = "IntermediateLayers04.h5"

scratch_cnn_dir = "./drive/My Drive/CNN"
scratch_train_cnn_model_name = "Dense01.h5"

SCCNN = models.load_model(os.path.join(scratch_cnn_dir, scratch_train_cnn_model_name))
PTCNN = models.load_model(os.path.join(pre_train_cnn_dir, pre_train_cnn_model_name))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
import numpy as np
from sklearn.model_selection import train_test_split

train_gray_imgs = np.load("/content/drive/My Drive/CI_FinalProject/Copia di train_img_224.npy")
train_lbls = np.load("drive/My Drive/CI_FinalProject/Copia di train_lab.npy")

val_split = 0.2
X_train, X_val, y_train, y_val = train_test_split(train_gray_imgs, train_lbls,
                                                  test_size=val_split, 
                                                  stratify=train_lbls)

X = np.stack((X_val,)*3, axis=-1)
y = y_val

del(X_train, y_train)
del(train_gray_imgs, train_lbls)

test_gray_imgs = np.load("drive/My Drive/CI_FinalProject/Copia di public_test_image_224.npy")
test_rgb_imgs = np.stack((test_gray_imgs,)*3, axis=-1)
test_lbls = np.load("drive/My Drive/CI_FinalProject/Copia di public_test_label.npy")

del(test_gray_imgs)

print(X.shape) # 2864 images: it is a small data set
print(X.dtype)

(573, 224, 224, 3)
uint16


In [0]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./65536, horizontal_flip=True, vertical_flip=True, validation_split=val_split)

val1_iterator = train_datagen.flow(X, y, shuffle=False ,batch_size=16, subset='validation')
val2_iterator = train_datagen.flow(X, y, shuffle=False ,batch_size=16, subset='validation')

# # validation_iterator = train_datagen.flow(train_rgb_imgs, train_lbls, batch_size=16, subset='validation')

test_datagen = ImageDataGenerator(rescale=1./65536, horizontal_flip=True, vertical_flip=True)

test_iterator = test_datagen.flow(test_rgb_imgs, test_lbls, batch_size=8)

In [32]:
PTCNNresults = PTCNN.evaluate_generator(val1_iterator, steps=35, verbose=1)
SCCNNresults = SCCNN.evaluate_generator(val2_iterator, steps=35, verbose=1)

print(PTCNNresults)
print(SCCNNresults)

PTCNNresults = PTCNN.evaluate_generator(test_iterator, steps=35, verbose=1)
SCCNNresults = SCCNN.evaluate_generator(test_iterator, steps=35, verbose=1)

print(PTCNNresults)
print(SCCNNresults)


35/35 [==============================] - 1s 24ms/step
[0.2944771457073868, 0.9489795918367347]
[0.33938139646637194, 0.8510204081632653]
35/35 [==============================] - 1s 16ms/step
[0.906763697122889, 0.8571428571428571]
[0.3546946481402431, 0.85]
